In [1]:
from tensorflow.keras.datasets import cifar10
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.applications.vgg19 import VGG19, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
import cv2
from keras_preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as BE
from tensorflow.keras.losses import categorical_crossentropy
from progressbar import ProgressBar
import os
import scipy.io
import tensorflow as tf
import json
import sys
sys.path.append('..')  
from GradPri_utils.utils import *

# from tensorflow.keras import backend as BE
# from Integrated_Gradients_algorithm import *
# from GradVisualizer import *

os.environ["CUDA_VISIBLE_DEVICES"] = "4"
config=tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess=tf.compat.v1.Session(config=config) 

# tf.compat.v1.disable_eager_execution()
# tf.compat.v1.enable_eager_execution()

In [2]:
base_model = load_model("./model/Udacity_VGG19.h5")

In [3]:
base_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_bn1 (BatchNormalizati (None, 128, 128, 64)      256       
_________________________________________________________________
block1_act1 (Activation)     (None, 128, 128, 64)      0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_bn2 (BatchNormalizati (None, 128, 128, 64)      256       
_________________________________________________________________
block1_act2 (Activation)     (None, 128, 128, 64)      0         
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0

In [5]:
DATA_PATH = '/public/liujiawei/ZHB/GradPri/UdacityData'

X_test = np.load(os.path.join(DATA_PATH, 'test_X.npy'))
Y_test = np.load(os.path.join(DATA_PATH, 'test_Y.npy'))

In [6]:
print('x_test shape:', X_test.shape)
print(X_test.shape[0], 'test samples')

x_test shape: (5279, 128, 128, 3)
5279 test samples


In [114]:
from functools import reduce
from art.utils import random_sphere
from art.config import ART_NUMPY_DTYPE
import sys
sys.path.append('..')  
from GradPri_utils.utils import *
from scipy.stats import weibull_min
from scipy.optimize import fmin as scipy_optimizer

def inverper_r(regression, x, nb_batches, batch_size, radius, norm, c_init=1.0, pool_factor=5):
    # Check if the targeted class is different from the predicted class
#     y_pred = regression.predict(np.array([x]))
    y_pred = regression.predict(np.reshape(x, [-1,128,128,3]))
#     pred_class = np.argmax(y_pred, axis=1)[0]

    # Check if pool_factor is smaller than 1
    if pool_factor < 1:  # pragma: no cover
        raise ValueError("The `pool_factor` must be larger than 1.")

    # Some auxiliary vars
    rand_pool_grad_set = []
    grad_norm_set = []
    dim = reduce(lambda x_, y: x_ * y, x.shape, 1)
    shape = [pool_factor * batch_size]
    shape.extend(x.shape)

    # Generate a pool of samples
    rand_pool = np.reshape(
        random_sphere(nb_points=pool_factor * batch_size, nb_dims=dim, radius=radius, norm=norm),
        shape,
    )
    rand_pool += np.repeat(np.array([x]), pool_factor * batch_size, 0)
    rand_pool = rand_pool.astype(ART_NUMPY_DTYPE)
#     if hasattr(regression, "clip_values") and regression.clip_values is not None:
    np.clip(rand_pool, -128, 128, out=rand_pool)

    # Change norm since q = p / (p-1)
    if norm == 1:
        norm = np.inf
    elif norm == np.inf:
        norm = 1
    elif norm != 2:  # pragma: no cover
        raise ValueError(f"Norm {norm} not supported")

    # Compute gradients for all samples in rand_pool
    for i in range(batch_size):
        rand_pool_batch = rand_pool[i * pool_factor : (i + 1) * pool_factor]

#         Compute gradients
#         grad_pred = regression.class_gradient(rand_pool_batch, label=pred_class)
        grad_pred=[]
        for x_tmp in rand_pool_batch:
            grads = get_gradients_regression_one_out(x_tmp, base_model)
            grad_pred.append(grads)
        if np.isnan(grad_pred).any() :  # pragma: no cover
            raise Exception("The regression results NaN gradients.")

        grad = grad_pred
        grad = np.reshape(grad, (pool_factor, -1))
        grad = np.linalg.norm(grad, ord=norm, axis=1)
        rand_pool_grad_set.extend(grad)

    rand_pool_grads = np.array(rand_pool_grad_set)

    # Loop over the batches
    for _ in range(nb_batches):
        # Random selection of gradients
        grad_norm = rand_pool_grads[np.random.choice(pool_factor * batch_size, batch_size)]
        grad_norm = np.max(grad_norm)
#         grad_norm = np.sum(np.abs(grad_norm)) 
        grad_norm_set.append(grad_norm)

    # Maximum likelihood estimation for max gradient norms
    [_, loc, _] = weibull_min.fit(-np.array(grad_norm_set), c_init, optimizer=scipy_optimizer)

    # Compute function value
#     values = regression.predict(np.reshape(x, [-1,128,128,3]))
    values = regression.predict(np.reshape(rand_pool_batch, [-1,128,128,3]))
#     value = np.min( [(1+prob_constant)*values[:, pred_class] , 1] ) - values[:, pred_class]
#     value = values[:, pred_class] - prob_constant
#     value = np.max( [(1+prob_constant)*values[:, pred_class] , 0.9] ) - values[:, pred_class]
#     value = values[0] * np.log(1+values[0])
#     value = np.max(np.sum(values, axis=-1)) - np.min(np.sum(values, axis=-1))

#     tmp1 = values[:, pred_class]*(1+prob_constant)
#     if tmp1<0.9:
#         value = 0.9-values[:, pred_class]
#     if tmp1>=0.9:
#         value = 1.001-values[:, pred_class]

    # Compute scores
#     score = np.min([-value[0] / loc, radius])
#     score = -value[0] / loc
    score = -1 / loc

    return score

In [121]:
R_L1 = 40
R_L2 = 2
R_LI = 20
top_set = 1 # 预测的前n个类中包含真实标签则表示预测正确
total_sample_num = 50
img_num = range(0, total_sample_num)
ground_truth_label = []  # 记录样本的真实标签
predicted_confidence = []  # 记录样本的置信度信息
BoundaryGini2 = []

pbar = ProgressBar()
for i in pbar(img_num):
#     img_path = DATA_PATH + file_name[i]
#     img = image.load_img(img_path, target_size=(224, 224))
#     x_tmp = image.img_to_array(img)
#     x_tmp = np.expand_dims(x_tmp, axis=0)
#     x_tmp = preprocess_input(x_tmp)
    
    x_tmp = X_test[i]
    x_tmp = np.expand_dims(x_tmp, axis=0)
    ground_truth_label.append(Y_test[i])
    predicted_confidence.append(base_model.predict(x_tmp)[0])
#     pre_tmp = base_model.predict(np.reshape(x_tmp, [-1,224,224,3]))
    res_tmp = inverper_r(regression=base_model, x=x_tmp, nb_batches=3, 
                         batch_size=10, radius=R_LI, norm=np.inf, c_init=1.0, pool_factor=5)
    BoundaryGini2.append(res_tmp)

(1,)
(2,)
(3,)


(1,)
(2,)
(3,)


(1,)
(2,)
(3,)
(1,)
(2,)
(3,)


(1,)
(2,)
(3,)
(1,)
(2,)
(3,)


(1,)
(2,)
(3,)


...

(1,)
(2,)
(3,)


(1,)
(2,)
(3,)


(1,)
(2,)
(3,)


(1,)
(2,)
(3,)
(1,)
(2,)
(3,)


100% |########################################################################|


In [217]:
num_metric = 50 #total_sample_num
thre_mse = None
indexs = np.argsort(BoundaryGini2[0: num_metric])
indexs = indexs[::-1]
top_set=1
# 计算APFD指标
APFD,_,_ = get_APFD_reg(Gini_indexs=indexs, ground_truth_label=np.array(ground_truth_label[0: num_metric]), 
                    predicted_confidence=np.array(predicted_confidence[0: num_metric]), thre_mse=0.0001)
print("APFD: ", APFD)

# 计算RAUC指标
RAUC,_,_ = get_RAUC_reg(Gini_indexs=indexs, ground_truth_label=np.array(ground_truth_label[0: num_metric]), 
                    predicted_confidence=np.array(predicted_confidence[0: num_metric]), thre_mse=0.0001)
print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.5129166666666667
RAUC:  0.9488993710691824


In [113]:
x_tmp = X_test[7]
x_tmp = np.expand_dims(x_tmp, axis=0)
grads = get_gradients_regression_one_out(x_tmp, base_model)
print(np.sum(np.abs(grads)))

0.12396701


In [191]:
indexs_rand = []
for i in range(len(indexs)):
    indexs_rand.append(i)
indexs_rand = np.array(indexs_rand)
indexs_rand
import random
random.shuffle(indexs_rand)

RAUC2,_,_ = get_RAUC_reg2(Gini_indexs=indexs_rand, ground_truth_label=np.array(ground_truth_label[0: num_metric]), 
                    predicted_confidence=np.array(predicted_confidence[0: num_metric]))
print("RAUC: ", RAUC, '\t', "RAUC2: ", RAUC2)

RAUC:  0.6340508806262231 	 RAUC2:  0.3056


array([40, 44,  4, 27,  3, 31, 15, 29, 19, 48, 41, 46, 45, 13, 34,  2, 35,
       36, 14, 17, 25, 16,  1, 26, 38, 28, 42, 24, 47, 11, 22, 43, 33, 23,
       30, 39,  8,  6, 21, 10, 37, 49,  0, 32,  5,  7,  9, 18, 12, 20])

In [116]:
def get_APFD_reg(Gini_indexs, ground_truth_label, predicted_confidence, thre_mse=None):
    o_i = 0
    pbar = ProgressBar()
    wrong_num = 0
    wrong_num_index = []
    
    if thre_mse is None:
        mse_tmp = []
        for i in range(len(Gini_indexs)):
            mse_tmp.append( np.sum(pow( predicted_confidence[i][0] - ground_truth_label[i], 2)))
        thre_mse = np.mean(mse_tmp)
        # thre_rmse = np.sqrt(thre_mse)

    for i in pbar(range(0, len(Gini_indexs))):
#         mse = np.sum(pow( ground_truth_label[i] - predicted_confidence[i][0], 2))
        mse = np.sum(pow( ground_truth_label[Gini_indexs[i]] - predicted_confidence[Gini_indexs[i]][0], 2))
        if mse > thre_mse:
            o_i = o_i+i
            wrong_num = wrong_num+1
            wrong_num_index.append(Gini_indexs[i])
    APFD = 1 - o_i/(len(Gini_indexs)*wrong_num) + 1/(2*len(Gini_indexs))
    return APFD, wrong_num, np.array(wrong_num_index).reshape(-1)

def get_RAUC_reg(Gini_indexs, ground_truth_label, predicted_confidence, thre_mse=None):
    pre_y_axis = []
    o_i = 0
    wrong_num = 0
    pbar = ProgressBar()
    
    if thre_mse is None:
        mse_tmp = []
        for i in range(len(Gini_indexs)):
            mse_tmp.append( np.sum(pow( predicted_confidence[i][0] - ground_truth_label[i], 2)))
        thre_mse = np.mean(mse_tmp)
    
    for i in pbar(range(0, len(Gini_indexs))):
        mse = np.sum(pow( ground_truth_label[Gini_indexs[i]] - predicted_confidence[Gini_indexs[i]][0], 2))
        if mse > thre_mse:
            o_i = o_i+1
            wrong_num = wrong_num+1
            pre_y_axis.append(o_i)
        else:
            pre_y_axis.append(o_i)
    true_y_axis = wrong_num*(len(Gini_indexs)-wrong_num) + (wrong_num+1)*wrong_num/2
    RAUC = np.sum(pre_y_axis)/true_y_axis
#     print("RAUC: ", RAUC)
    return RAUC, len(Gini_indexs), wrong_num

In [169]:
def get_RAUC_reg2(Gini_indexs, ground_truth_label, predicted_confidence):
    pre_y_axis = []
    o_i = 0
    wrong_num = None
    pbar = ProgressBar()

    mse_tmp = []
    for i in range(len(Gini_indexs)):
        mse_tmp.append( np.sum(pow( predicted_confidence[i][0] - ground_truth_label[i], 2)))
    indexs = np.argsort(mse_tmp)
    indexs = indexs[::-1]
#     print(mse_tmp, indexs)
    
    tmp1 = 0
    for i in pbar(range(len(Gini_indexs))):
        for j in range(i+1):
            if Gini_indexs[i]==indexs[j]:
                tmp1 += 1
        pre_y_axis.append(tmp1)
#     print(pre_y_axis)
    true_y_axis = len(Gini_indexs)*len(Gini_indexs)/2
    RAUC = np.sum(pre_y_axis)/true_y_axis
#     true_y_axis = wrong_num*(len(Gini_indexs)-wrong_num) + (wrong_num+1)*wrong_num/2
#     RAUC = np.sum(pre_y_axis)/true_y_axis
# #     print("RAUC: ", RAUC)
    return RAUC, len(Gini_indexs), wrong_num